In [1]:
import pandas as pd
import scipy as sc
import numpy as np
import sklearn
import pickle
import pathlib as Path
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sb
sb.set(font='IPAexGothic')

import multiprocessing
import itertools
import collections
import datetime
import gc

from tqdm._tqdm_notebook import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)

data_path = Path.Path("../data")
result_path = Path.Path("../result")

prefix = 'ana003'

# データ抽出
> 全体データ（all_df.pkl） <br>

In [2]:
all_df = pd.read_pickle(data_path / 'all_df.pkl')

In [3]:
all_df

,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y
id,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,False,False,False,内角低め,17.0,J,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,1,0.0
1,1,0,0,False,False,False,内角低め,14.0,I,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,ストレート,今永 昇太,L,149km/h,2,1.0
2,1,0,1,False,False,False,外角高め,8.0,D,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,チェンジアップ,今永 昇太,L,137km/h,3,0.0
3,2,0,1,False,False,False,内角中心,21.0,G,ピレラ,R,NaN,NaN,NaN,1,20202173,1回表,NaN,スライダー,今永 昇太,L,138km/h,4,2.0
4,2,0,2,False,False,False,外角中心,7.0,F,ピレラ,R,G,S,38.3,1,20202173,1回表,False,チェンジアップ,今永 昇太,L,136km/h,5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33803,0,0,1,False,False,False,NaN,NaN,NaN,坂口 智隆,L,NaN,NaN,NaN,0,20202023,5回表,NaN,NaN,床田 寛樹,L,NaN,2,NaN
33804,0,0,0,False,False,False,NaN,NaN,NaN,メヒア,R,NaN,NaN,NaN,0,20202640,9回表,NaN,NaN,堀岡 隼人,R,NaN,1,NaN
33805,0,0,0,True,False,False,NaN,NaN,NaN,鈴木 誠也,R,NaN,NaN,NaN,0,20202864,7回裏,NaN,NaN,ディプラン,R,NaN,1,NaN


# 列定義
>column name example	description	exist on test data? <br>
>totalPitchingCount	各打席の投球が何球目か	yes<br>
>B	ボールカウント	yes<br>
>S	ストライクカウント	yes<br>
>O	アウトカウント	yes<br>
>b1	1塁にランナーがいるか	yes<br>
>b2	2塁にランナーがいるか	yes<br>
>b3	3塁にランナーがいるか	yes<br>
>pitcher ピッチャー名（NaNを含む）	yes<br>
>pitcherHand 右投げか左投げか	yes<br>
>batter	バッター名（NaNを含む）	yes<br>
>batterHand	右打ちか左打ちか	yes<br>
>gameID	試合ID	yes<br>
>inning	イニング	yes<br>
>pitchType 球種	no<br>
>speed 投球速度	no<br>
>ballPositionLabel 投球位置	no<br>
>ballX 投球のX座標(1-21)	no<br>
>ballY 投球のY座標(A-K)	no<br>
>dir 打球方向(A-Z)	no<br>
>dist 打球距離	no<br>
>battingType 打球種類(B:バント, G:ゴロ, P:ポップフライ, F:フライ, L:ライナー)	no<br>
>isOuts 投球結果がアウトか	no<br>
>y 投球結果(0:ボール, 1:ストライク, 2:ファウル, 3:アウト, 4:シングルヒット, 5:二塁打, 6:三塁打, 7:ホームラン)	<br>

# 欠損確認

In [10]:
all_df.isnull().sum().to_frame(name='欠損数')

,欠損数
B,0
O,0
S,0
b1,0
b2,0
b3,0
ballPositionLabel,33808
ballX,33808
ballY,33808
batter,5


> ・バッター、ピッチャーが欠損ってなに？ <br>
> ・inningとgame_idがあれば、バッターおよびピッチャーのチームは特定可能

In [13]:
# バッターが欠損のデータ
all_df[all_df['batter'].isnull()]

,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y
id,,,,,,,,,,,,,,,,,,,,,,,,
7370,1,1,0,False,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20202836,9回裏,NaN,NaN,益田 直也,NaN,NaN,2,NaN
12016,0,2,1,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20202114,1回裏,NaN,NaN,Ｋ．ジョンソン,NaN,NaN,2,NaN
14797,1,2,0,False,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20202468,6回表,NaN,NaN,野村 祐輔,NaN,NaN,2,NaN
18474,0,2,0,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20202413,9回裏,NaN,NaN,三嶋 一輝,NaN,NaN,1,NaN
19000,0,2,0,True,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,20202774,7回裏,NaN,NaN,漆原 大晟,NaN,NaN,1,NaN


In [14]:
# ピッチャーが欠損
all_df[all_df['pitcher'].isnull()]

,B,O,S,b1,b2,b3,ballPositionLabel,ballX,ballY,batter,batterHand,battingType,dir,dist,flg_train,gameID,inning,isOuts,pitchType,pitcher,pitcherHand,speed,totalPitchingCount,y
id,,,,,,,,,,,,,,,,,,,,,,,,
12497,0,0,0,False,False,False,NaN,NaN,NaN,シエラ,R,NaN,NaN,NaN,0,20202814,8回表,NaN,NaN,NaN,R,NaN,1,NaN


# 簡易集計
> イニング×出塁状態×結果（訓練データ）

In [21]:
all_df[
        # 訓練データのみ
        all_df['flg_train']==1
    
    ].groupby(
        # イニング×出塁状態×結果
        ['inning', 'b1', 'b2', 'b3', 'y']

    # パターンが多いので外部ファイルに出力
    )[['B', 'S', 'O']].mean().to_csv(data_path / ('%s_freq001.csv' % prefix))